In [1]:
%%time
from tqdm import tqdm
import pandapower as pp
import pandas as pd
import numpy as np
import pandapower.converter as pc
from pandapower.pypower.makePTDF import makePTDF
import pyomo.environ as pyo
from scipy.sparse import lil_matrix
from matpowercaseframes import CaseFrames
matpowercaseframes_imported = True

CPU times: total: 3.45 s
Wall time: 4.64 s


In [2]:
%%time
mpc_file = 'data\\pglib-opf-17.08\\pglib_opf_case1888_rte.m'
mpc_frames = CaseFrames(mpc_file)
ppc = {key: mpc_frames.__getattribute__(key) if not isinstance(
    mpc_frames.__getattribute__(key), pd.DataFrame) else mpc_frames.__getattribute__(
    key).values for key in mpc_frames._attributes}
baseMVA, bus, branch, gen = ppc['baseMVA'], ppc['bus'], ppc['branch'], ppc['gen']
bus_index_dict = {}
for i, index in enumerate(ppc['bus'][:,0]):
    bus_index_dict[index.astype(int)] = i
baseMVA,bus,branch = ppc['baseMVA'],ppc['bus'],ppc['branch']
for i,bus_name in enumerate(bus[:,0]):
    bus[i,0] = bus_index_dict[bus_name]
for i,fbus_name in enumerate(branch[:,0]):
    branch[i,0] = bus_index_dict[fbus_name]
for i,tbus_name in enumerate(branch[:,1]):
    branch[i,1] = bus_index_dict[tbus_name]
for i,gen_name in enumerate(gen[:,0]):
    gen[i,0] = bus_index_dict[gen_name]    
gen_index = gen[:,0].astype(int)
bus_index = bus[:,0].astype(int)
H = lil_matrix((len(bus_index), len(gen_index)), dtype=int)
for col, gen_bus in enumerate(gen_index):
    H[gen_bus, col] = 1
H = H.tocsr()               
M = makePTDF(baseMVA, bus, branch)
f_max = np.array([rateA for i in range(len(branch)) for j,rateA in enumerate(branch[i]) if j == 5])
# f_max = np.array([999999 for i in range(len(branch)) for j,rateA in enumerate(branch[i]) if j == 5])
p_init = mpc_frames.gen.PG.values
d = mpc_frames.bus.PD.values
sigma_scaling = 0.03
sigma = (sigma_scaling * d)
mean = np.zeros(len(d))
covariance_matrix = np.diag(sigma**2)
num_samples = 1
omega = np.random.multivariate_normal(mean, covariance_matrix, num_samples)
gen_lb = mpc_frames.gen.PMIN.values
gen_ub = mpc_frames.gen.PMAX.values
nbus = len(mpc_frames.bus)
nline = len(mpc_frames.branch)
cost_coeff = mpc_frames.gencost.loc[:,'COST_2':'COST_0'].values

CPU times: total: 18.7 s
Wall time: 2.15 s


In [3]:
%%time
# paper model
model = pyo.ConcreteModel()
# sets
ngen = len(gen_index)
model.ng = pyo.RangeSet(0, ngen-1)
nbus = len(bus_index)
model.nb = pyo.RangeSet(0, nbus-1)
nline = len(ppc['branch'])
model.nl = pyo.RangeSet(0, nline-1)
# variables
model.p = pyo.Var(model.ng, domain=pyo.Reals, initialize=p_init)
model.omega = pyo.Var(model.nb, domain=pyo.Reals)
# objective (1a)
model.obj = pyo.Objective(expr =
                    cost_coeff[:,0].sum() + sum([cost_coeff[:,1][g]*model.p[g] for g in model.ng])\
                    + sum([cost_coeff[:,2][g]*model.p[g] for g in model.ng])
                    , sense = pyo.minimize
)
# constraint (1b) μ is omitted
model.c1b = pyo.Constraint(expr =
                          sum([model.p[g] for g in model.ng ]) == 
                           sum([d[b] for b in model.nb]) 
                           - sum([model.omega[b] for b in model.nb])
                          )
# constraint (1c)
def c1c_lb_rule(model, g):
    return gen_lb[g] <= model.p[g]
def c1c_ub_rule(model, g):
    return gen_ub[g] >= model.p[g]
model.c1c_lb = pyo.Constraint(model.ng, rule=c1c_lb_rule)
model.c1c_ub = pyo.Constraint(model.ng, rule=c1c_ub_rule)
# constraint (1d) μ is omitted
flow_expr = M@(H.toarray()@model.p + model.omega - (d))
def c1d_lb_rule(model, l):
    return -f_max[l] <= flow_expr[l]
def c1d_ub_rule(model, l):
    return f_max[l] >= flow_expr[l]
model.c1d_lb = pyo.Constraint(model.nl, rule=c1d_lb_rule)
model.c1d_ub = pyo.Constraint(model.nl, rule=c1d_ub_rule)

CPU times: total: 17.8 s
Wall time: 16.4 s


In [ ]:
%%time
solver = pyo.SolverFactory('mosek')
obj_dict={}
infeasible=0
for i, value in tqdm(enumerate(omega)):
    for j in model.nb:
        model.omega[j].fix(value[j]) #
    res = solver.solve(model)
    if not res.solver.termination_condition == 'optimal':
        infeasible+=1
#         log_infeasible_constraints(instance)
    obj_dict[i]=pyo.value(model.obj)
print(f'C_total=', pyo.value(model.obj), 'infeasible=', infeasible)
# for i in instance.p:
#     print(f'P{i}=', pyo.value(instance.p[i]))

0it [00:00, ?it/s]

In [6]:
C_total= 1487402.4983541125 infeasible= 0

SyntaxError: invalid syntax (3859781387.py, line 1)

In [ ]:
# from pyomo.util.infeasible import log_infeasible_constraints
# import logging
# logging.basicConfig(level=logging.INFO)
# log_infeasible_constraints(model)

# Learn

In [ ]:
# # success:
# net = pc.from_mpc(mpc_file)
# pp.runpp(net)

In [ ]:
# makeBdc.py

# from numpy import ones, zeros_like, r_, pi, flatnonzero as find, real, int64, float64, divide, errstate 
# from pandapower.pypower.idx_brch import F_BUS, T_BUS, BR_X, TAP, SHIFT, BR_STATUS
# from pandapower.pypower.idx_bus import BUS_I
# from scipy.sparse import csr_matrix, csc_matrix
# try:
#     import pandaplan.core.pplog as logging
# except ImportError:
#     import logging
# logger = logging.getLogger(__name__)    

In [ ]:
# return_csr = True
# sparse = csr_matrix if return_csr else csc_matrix
# nb = bus.shape[0]          ## number of buses
# nl = branch.shape[0]       ## number of lines
# stat = real(branch[:, BR_STATUS])  ## ones at in-service branches
# br_x = real(branch[:, BR_X])  ## ones at in-service branches
# b = zeros_like(stat, dtype=float64)
# divide(stat, br_x, out=b, dtype=float64)  ## series susceptance
# tap = ones(nl)  ## default tap ratio = 1
# i = find(t := real(branch[:, TAP]))  ## indices of non-zero tap ratios
# tap[i] = t[i]  ## assign non-zero tap ratios
# tap
# b = b / tap
# ## build connection matrix Cft = Cf - Ct for line and from - to buses
# f = real(branch[:, F_BUS]).astype(int64)                           ## list of "from" buses
# t = real(branch[:, T_BUS]).astype(int64)                           ## list of "to" buses
# i = r_[range(nl), range(nl)]                   ## double set of row indices

In [ ]:
# ## connection matrix
# Cft = sparse((r_[ones(nl), -ones(nl)], (i, r_[f, t])), (nl, nb))
# ## build Bf such that Bf * Va is the vector of real branch powers injected
# ## at each branch's "from" bus
# Bf = sparse((r_[b, -b], (i, r_[f, t])), (nl, nb))## = spdiags(b, 0, nl, nl) * Cft